## Basic speed test using 'time' module

In [ ]:
def code():
    ...
    ...

def speed_test():
    import time
    
    start=time.time()
    code()
    end=time.time() - start
    print("Total time taken: ",end)

## Macro-profiling

### Save the code in a file and run the program as:
#### <font color='blue'>python3 -m cProfile testcode.py</font>

In [ ]:
import time
def func1(): # Function 1
    time.sleep(0.01)

def func2(): # Function 2
    time.sleep(0.05)

def main():
    for i in range(50):
        func1()
        func2()
    
if '__name__'== '__main__':
    main()

#### Output will have metrics as:
<p><font color='green'>ncals</font> : Number of calls.<br>
<font color='green'>tottime</font> : Total time spent in the function.<br>
<font color='green'>percall</font> : Time per call (tottime/ncalls).<br>
<font color='green'>cumtime</font> : Cumulative time spent in a function.<br>
<font color='green'>time per call</font> : Quotient of cumtime over the number of primitive calls.<br>
<font color='green'>filename:lineno</font> : File name and the line number of the code.<br></p>

#### Using <font color='blue'>cProfile</font> inside a program

In [12]:
import cProfile

from testcode import main # importing function from our program
cProfile.run('main()', 'myapp.stats') # report saved in a file called 'myapp.stats'

#### Using <font color='blue'>pstats</font> module to see the metrics from cProfile/profile

In [14]:
import pstats #analyzes data collected by the Python profiler

stats = pstats.Stats('myapp.stats') # Supply path of the data
stats.sort_stats('time', 'name').print_stats()

Sun May  3 12:50:26 2020    myapp.stats

         404 function calls in 60.632 seconds

   Ordered by: internal time, function name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200   60.628    0.303   60.628    0.303 {built-in method time.sleep}
        1    0.001    0.001   60.631   60.631 /Users/pranaygore/testcode.py:9(main)
      100    0.001    0.000   50.306    0.503 /Users/pranaygore/testcode.py:6(func2)
      100    0.001    0.000   10.324    0.103 /Users/pranaygore/testcode.py:3(func1)
        1    0.000    0.000   60.632   60.632 {built-in method builtins.exec}
        1    0.000    0.000   60.632   60.632 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




#### Find metrics related to a specific function of your code

In [21]:
stats.print_callees('func1')

   Ordered by: internal time, function name
   List reduced from 7 to 1 due to restriction <'func1'>

Function                                called...
                                            ncalls  tottime  cumtime
/Users/pranaygore/testcode.py:3(func1)  ->     100   10.324   10.324  {built-in method time.sleep}




#### Install <font color='blue'>snakeviz</font> module for visualization

In [44]:
!conda install -c anaconda snakeviz --yes

Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - snakeviz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    snakeviz-2.0.1             |           py37_0         315 KB  anaconda
    ------------------------------------------------------------
                                           Total:         315 KB

The following NEW packages will be INSTALLED:

  snakeviz           anaconda/osx-64::snakeviz-2.0.1-py37_0



snakeviz-2.0.1       | 315 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [47]:
!snakeviz myapp.stats

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fpranaygore%2Fmyapp.stats
^C

Bye!


## Micro-profiling

In [48]:
import tempfile, os, cProfile, pstats
from testcode import main as main_profile # import the main function from our testcode.

def profile(column='time', list=5): # Creating a decorator function
    def _profile(function):
        def __profile(*args, **kw):
            s = tempfile.mktemp()
            profiler = cProfile.Profile()
            profiler.runcall(function, *args, **kw)
            profiler.dump_stats(s)
            p = pstats.Stats(s)
            p.sort_stats(column).print_stats(list)
        return __profile
    return _profile

@profile('time', 6)
def main():
    return main_profile()# Calling main function of testcode in current programs main

main()

Sun May  3 20:43:07 2020    /var/folders/3v/nxmqphns1tx7_q_1pqn11v2w0000gn/T/tmp37m6f45d

         403 function calls in 60.667 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200   60.664    0.303   60.664    0.303 {built-in method time.sleep}
        1    0.001    0.001   60.667   60.667 /Users/pranaygore/testcode.py:9(main)
      100    0.001    0.000   10.349    0.103 /Users/pranaygore/testcode.py:3(func1)
      100    0.001    0.000   50.317    0.503 /Users/pranaygore/testcode.py:6(func2)
        1    0.000    0.000   60.667   60.667 <ipython-input-48-b1f95de56f2d>:17(main)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [50]:
# Profile a specific function and get the stats
from testcode import func1
stats_func1=profile()(func1)
stats_func1()

Sun May  3 20:49:35 2020    /var/folders/3v/nxmqphns1tx7_q_1pqn11v2w0000gn/T/tmplw5516lu

         3 function calls in 0.104 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.104    0.104    0.104    0.104 {built-in method time.sleep}
        1    0.000    0.000    0.104    0.104 /Users/pranaygore/testcode.py:3(func1)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




#### using <font color='blue'>timeit</font> module

In [56]:
import timeit
t=timeit.Timer('from testcode import func1;func1()') # complete code(to be tested) has to be in a string format
t.timeit(number=10) # number refers to no of times we need to execute.
# Result is the time taken to execute the code 10 times.

1.0417677559998992

#### We can also run <font color='blue'>timeit</font> in the terminal directly.

In [57]:
!python3 -m timeit '"-".join(map(str, range(100)))'

20000 loops, best of 5: 19.9 usec per loop


#### <font color='blue'>LineProfiler</font> module

In [60]:
!conda install -c anaconda line_profiler --yes

Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - line_profiler


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    line_profiler-2.1.2        |   py37h1de35cc_0          55 KB  anaconda
    ------------------------------------------------------------
                                           Total:          55 KB

The following NEW packages will be INSTALLED:

  line_profiler      anaconda/osx-64::line_profiler-2.1.2-py37h1de35cc_0



line_profiler-2.1.2  | 55 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [70]:
from line_profiler import LineProfiler
  
def test_func(num):
    for i in range(100):
        num+=i
    print(num)

profile_data = LineProfiler(test_func(50)) 
profile_data.print_stats()

5000
Timer unit: 1e-06 s



//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Could not extract a code object for the object None
  
